In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models_torch import *
import torch.optim as optim
from random import shuffle

In [3]:
gra_sharp = 2.
model = ViT_ST_ST_Compact3_TDC_gra_sharp(image_size=(160,128,128), patches=(4,4,4), dim=96, ff_dim=144, num_heads=4, num_layers=12, dropout_rate=0.1, theta=0.7).cuda()
model.load_state_dict(torch.load('../weights/Physformer_VIPL.pkl'))
model.train()
def physformer(x, model=model):
    with torch.no_grad():
        bvp = model(torch.tensor(x[..., [2, 1, 0]].transpose(0, 4, 1, 2, 3)).cuda().float()*2-1, gra_sharp)[0].cpu().detach().numpy()
        bvp[:,:30] = bvp[:,130:] = bvp[:, 30:130].mean(axis=-1).reshape(-1, 1)
        return bvp - bvp.mean(axis=-1).reshape(-1, 1)

In [5]:
eval_on_dataset(test_set_PURE, physformer, 160, (128, 128), step=2, batch=4, save='../results/PhysFormer_VIPL_PURE.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysFormer_VIPL_PURE.h5')

100%|██████████| 59/59 [05:02<00:00,  5.12s/it]


{'Sliding window': {'MAE': 11.218, 'RMSE': 21.947, 'R': 0.62895},
 'Whole video': {'MAE': 11.12, 'RMSE': 21.757, 'R': 0.63952}}

In [4]:
eval_on_dataset(test_set_UBFC_rPPG2, physformer, 160, (128, 128), step=2, batch=4, save='../results/PhysFormer_VIPL_UBFC.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysFormer_VIPL_UBFC.h5')

100%|██████████| 42/42 [03:25<00:00,  4.90s/it]


{'Sliding window': {'MAE': 1.382, 'RMSE': 3.562, 'R': 0.97855},
 'Whole video': {'MAE': 0.896, 'RMSE': 1.263, 'R': 0.99735}}